# Deepfake Detection Model

Detecting deepfake images using finetuned XceptionNet

# 1. Import Modules

In [1]:
import torch
from torch import nn
import matplotlib as plt
import numpy as np
import pandas as pd
import pandas as pd
import os
from torch.utils.data import dataset, dataloader
from torchvision import datasets, models, transforms
import torchvision.transforms as transforms
import sklearn
import timm
from torchsummary import summary
from pathlib import Path
from PIL import Image
import random
import shutil
from sklearn.model_selection import train_test_split
import torchinfo
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from torch.optim.lr_scheduler import ReduceLROnPlateau

c:\NTU\Modules\Y1S2\CC0002\Codes\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. Setup Images

In [9]:
image_path = Path("DeepFakeDetection/") / "DeepFakeDetection"

In [10]:
image_path_list = list(image_path.glob("*/*.jpg"))

In [11]:
len(image_path_list)

140000

## Image Pre-Processing

### Splitting train test images

fake is 1, real is 0

In [2]:
fake_dir = os.path.join(os.getcwd(), "DeepFakeDetection", "DeepFakeDetection", "fake")
real_dir = os.path.join(os.getcwd(), "DeepFakeDetection", "DeepFakeDetection", "real")

fake_paths = [os.path.join(fake_dir, f) for f in os.listdir(fake_dir)]
real_paths = [os.path.join(real_dir, f) for f in os.listdir(real_dir)]

filepaths = fake_paths + real_paths
labels = ["fake"] * len(fake_paths) + ["true"] * len(real_paths)
train_paths, test_paths, train_labels, test_labels = train_test_split(
    filepaths,
    labels,
    test_size=0.1,         # 10% for test, adjust as needed
    stratify=labels,       # ensures class balance in splits
    random_state=42
)

In [13]:
def copy_files(paths, labels, split="train"):
    for path, label in zip(paths, labels):
        # e.g., 'images/train/fake/'
        out_dir = os.path.join("images", split, label)
        os.makedirs(out_dir, exist_ok=True)
        
        # e.g., 'images/train/fake/image123.jpg'
        filename = os.path.basename(path)
        out_path = os.path.join(out_dir, filename)
        
        # Copy the file
        shutil.copy2(path, out_path)

In [14]:
copy_files(train_paths, train_labels, split="train")
copy_files(test_paths, test_labels, split="test")

### Image Augmentation

First, we need to normalize the images

In [15]:
train_transforms = transforms.Compose([
    transforms.Resize((299, 299)),        # e.g., for Inception-like networks
    transforms.RandomHorizontalFlip(),    # flips half the images horizontally
    transforms.RandomRotation(10),        # random rotation ±10 degrees
    transforms.ToTensor(),                # convert to PyTorch
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

### Datasets and Dataloader

In [16]:
train_dir = os.path.join(os.getcwd(), "images", "train")
test_dir = os.path.join(os.getcwd(), "images", "test")

In [17]:
# Create Dataset objects
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transforms)
test_dataset  = datasets.ImageFolder(root=test_dir, transform=test_transforms)

# Create DataLoaders
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Train samples: {len(train_dataset)}, Test samples: {len(test_dataset)}")

Train samples: 126000, Test samples: 14000


# 3. Setup Models

In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device", device)

Using device cpu


In [19]:
model = timm.create_model('xception', pretrained=True, num_classes=2)

c:\NTU\Modules\Y1S2\CC0002\Codes\venv\Lib\site-packages\timm\models\_factory.py:126: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


In [20]:
model.to(device)

Xception(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act2): ReLU(inplace=True)
  (block1): Block(
    (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (rep): Sequential(
      (0): SeparableConv2d(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): SeparableConv2d(
        (conv1): Conv

In [21]:
summary(model, (3, 299, 299))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 149, 149]             864
       BatchNorm2d-2         [-1, 32, 149, 149]              64
              ReLU-3         [-1, 32, 149, 149]               0
            Conv2d-4         [-1, 64, 147, 147]          18,432
       BatchNorm2d-5         [-1, 64, 147, 147]             128
              ReLU-6         [-1, 64, 147, 147]               0
            Conv2d-7         [-1, 64, 147, 147]             576
            Conv2d-8        [-1, 128, 147, 147]           8,192
   SeparableConv2d-9        [-1, 128, 147, 147]               0
      BatchNorm2d-10        [-1, 128, 147, 147]             256
             ReLU-11        [-1, 128, 147, 147]               0
           Conv2d-12        [-1, 128, 147, 147]           1,152
           Conv2d-13        [-1, 128, 147, 147]          16,384
  SeparableConv2d-14        [-1, 128, 1

In [22]:
# Print a summary using torchinfo (uncomment for actual output)
torchinfo.summary(model=model, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
) 

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
Xception (Xception)                      [32, 3, 224, 224]    [32, 2]              --                   True
├─Conv2d (conv1)                         [32, 3, 224, 224]    [32, 32, 111, 111]   864                  True
├─BatchNorm2d (bn1)                      [32, 32, 111, 111]   [32, 32, 111, 111]   64                   True
├─ReLU (act1)                            [32, 32, 111, 111]   [32, 32, 111, 111]   --                   --
├─Conv2d (conv2)                         [32, 32, 111, 111]   [32, 64, 109, 109]   18,432               True
├─BatchNorm2d (bn2)                      [32, 64, 109, 109]   [32, 64, 109, 109]   128                  True
├─ReLU (act2)                            [32, 64, 109, 109]   [32, 64, 109, 109]   --                   --
├─Block (block1)                         [32, 64, 109, 109]   [32, 128, 55, 55]    --                   True
│    └─Sequential 

We need to freeze all layers except for the last

In [23]:
# Freeze all parameters
for param in model.parameters():
    param.requires_grad = False

# Unfreeze parameters in the final fc layer
for param in model.fc.parameters():
    param.requires_grad = True


### Training Arguments

In [24]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
num_epochs = 30
patience = 3
best_loss = np.inf
best_acc = 0
patience_counter = 0

c:\NTU\Modules\Y1S2\CC0002\Codes\venv\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [25]:
model_path = "trained_model.pth"

### Training

In [26]:
# Initialize dictionary to store metrics for every batch
metrics = {
    "train_loss": [],
    "train_acc": [],
    "test_loss": [],
    "test_acc": []
}

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0
    
    # Training phase with progress bar and per-batch metrics logging
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
    for images, labels in pbar:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # Calculate batch metrics
        batch_loss = loss.item()
        _, preds = torch.max(outputs, 1)
        batch_acc = (preds == labels).float().mean().item()
        
        # Append batch metrics to dictionary
        metrics["train_loss"].append(batch_loss)
        metrics["train_acc"].append(batch_acc)
        
        # Update running totals for epoch-level summary
        train_loss += batch_loss * images.size(0)
        total_train += labels.size(0)
        correct_train += (preds == labels).sum().item()
        
        # Update progress bar
        current_acc = correct_train / total_train
        pbar.set_postfix({'loss': f"{batch_loss:.4f}", 'train_acc': f"{current_acc:.4f}"})
    
    avg_train_loss = train_loss / total_train
    train_acc = correct_train / total_train

    # Early stopping check (inside your training loop)
    if avg_test_loss < best_loss:
        best_loss = avg_test_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break


    # Evaluation phase: record per-batch test metrics too
    model.eval()
    test_loss = 0.0
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            batch_loss = loss.item()
            _, preds = torch.max(outputs, 1)
            batch_acc = (preds == labels).float().mean().item()
            
            # Append batch metrics for test data
            metrics["test_loss"].append(batch_loss)
            metrics["test_acc"].append(batch_acc)
            
            test_loss += batch_loss * images.size(0)
            total_test += labels.size(0)
            correct_test += (preds == labels).sum().item()
    
    avg_test_loss = test_loss / total_test
    test_acc = correct_test / total_test

    print(f"Epoch [{epoch+1}/{num_epochs}] | "
          f"Train Loss: {avg_train_loss:.6f}, Train Acc: {train_acc:.6f} | "
          f"Test Loss: {avg_test_loss:.6f}, Test Acc: {test_acc:.6f}")
    
    if test_acc > best_acc:
        best_acc = test_acc
        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_acc': best_acc,
        }
        # Save the model's state dictionary
        torch.save(model.state_dict(), model_path)
        print(f"Checkpoint saved at epoch {epoch+1} with test accuracy: {test_acc:.6f}")

    scheduler.step(avg_test_loss)

# After training, the 'metrics' dictionary holds per-batch values for plotting.


KeyboardInterrupt: 

## Visualization

### Plotting Accuracy and Loss

In [ ]:
# Create a figure with two subplots: one for loss and one for accuracy
plt.figure(figsize=(12, 5))

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(metrics["train_loss"], label="Train Loss", alpha=0.7)
plt.plot(metrics["test_loss"], label="Test Loss", alpha=0.7)
plt.xlabel("Batch Number")
plt.ylabel("Loss")
plt.title("Loss vs. Batches")
plt.legend()

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(metrics["train_acc"], label="Train Accuracy", alpha=0.7)
plt.plot(metrics["test_acc"], label="Test Accuracy", alpha=0.7)
plt.xlabel("Batch Number")
plt.ylabel("Accuracy")
plt.title("Accuracy vs. Batches")
plt.legend()

plt.tight_layout()
plt.show()

### Confusion Matrix

In [ ]:
# Define a helper function to evaluate the model and gather true and predicted labels.
def evaluate_model(model, dataloader, device):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            # Get predicted class (the index with the highest logit)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    return np.array(y_true), np.array(y_pred)

# Compute metrics for a given set of true and predicted labels.
def compute_metrics(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    # For binary classification, you can use average="binary" if your positive class is 1.
    # Here, to keep it general, we use "weighted".
    prec = precision_score(y_true, y_pred, average="weighted", zero_division=0)
    rec = recall_score(y_true, y_pred, average="weighted", zero_division=0)
    f1 = f1_score(y_true, y_pred, average="weighted", zero_division=0)
    return cm, acc, prec, rec, f1

# Define a function to plot the confusion matrix using matplotlib.
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(6, 5))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    # Annotate the confusion matrix with counts.
    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in np.ndindex(cm.shape):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

# Evaluate on training data
train_y_true, train_y_pred = evaluate_model(model, train_loader, device)
train_cm, train_acc, train_prec, train_rec, train_f1 = compute_metrics(train_y_true, train_y_pred)

print("Training Metrics:")
print(f"Accuracy: {train_acc:.4f}")
print(f"Precision: {train_prec:.4f}")
print(f"Recall: {train_rec:.4f}")
print(f"F1 Score: {train_f1:.4f}")

# Evaluate on test data
test_y_true, test_y_pred = evaluate_model(model, test_loader, device)
test_cm, test_acc, test_prec, test_rec, test_f1 = compute_metrics(test_y_true, test_y_pred)

print("\nTest Metrics:")
print(f"Accuracy: {test_acc:.4f}")
print(f"Precision: {test_prec:.4f}")
print(f"Recall: {test_rec:.4f}")
print(f"F1 Score: {test_f1:.4f}")

# Plot the confusion matrices.
plot_confusion_matrix(train_cm, classes=["Fake", "Real"], title="Training Confusion Matrix")
plot_confusion_matrix(test_cm, classes=["Fake", "Real"], title="Test Confusion Matrix")
plt.show()